#Cài đặt thư viện

In [ ]:
!pip install -q transformers==4.41.0
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.2
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4
!pip install -q rapidocr-onnxruntime==1.4.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.1 MB/s 

In [ ]:
!pip install --upgrade transformers

#Import thư viện

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from typing import List

#Chuẩn bị dữ liệu

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pdf_files: List[str]  = [
    "/content/drive/My Drive/Colab/RAG/cnxhkh.pdf",
    "/content/drive/My Drive/Colab/RAG/lsd.pdf",
    "/content/drive/My Drive/Colab/RAG/tthcm.pdf",
    "/content/drive/My Drive/Colab/RAG/thmaclenin.pdf"
]

In [ ]:
def remove_non_utf8_characters(text):
    utf8_encoded_text = ""
    for char in text:
        try:
            char.encode('utf-8')
            utf8_encoded_text += char
        except UnicodeEncodeError:
            pass
    return utf8_encoded_text

In [ ]:
documents = []

for pdf_file in pdf_files:
    docs = PyPDFLoader(pdf_file, extract_images=True).load()
    for doc in docs:
        doc.page_content = remove_non_utf8_characters(doc.page_content)
    documents.extend(docs)

/usr/local/lib/python3.10/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (130160568 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [ ]:
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000,
                                                            chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print("Number of sub-documents: ", len(docs))
print(docs[1])

Number of sub-documents:  308
page_content='Mục lục
1 CNXHKH là gì? Hoàn cảnh lịch sử ra đời chủ nghĩa xã hội khoa học . . . . . . . 3
2 Vai trò của C.Mác và Ph. Ăngghen đối với sự ra đời của chủ nghĩa xã hội khoa
học. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6
3 Những điều kiện khách quan và chủ quan quy định sứ mệnh lịch sử của
GCCN . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8
4 Tính tất yếu khách quan của thời kỳ quá độ lên chủ nghĩa xã hội? . . . . . . . . 10
5 Khái niệm dân tộc và cương lĩnh dân tộc của chủ nghĩa Mác – Lênin. . . . . 12
6 Quan điểm của chủ nghĩa Mác – Lênin về bản chất, nguồn gốc và nguyên tắc
giải quyết vấn đề tôn giáo trong thời kỳ quá độ lên CNXH . . . . . . . . . . . . . . . . 14
7 Phân tích các chức năng cơ bản của gia đình . . . . . . . . . . . . . . . . . . . . . . . . . . 17'

#Embedding và lưu vào Db

In [ ]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vector_db.as_retriever()

In [ ]:
result = retriever.invoke("Chủ nghĩa xã hội khoa học là gì?")
print("Number of relevant documents: ", len (result))

Number of relevant documents:  4


#Model

In [ ]:
nf4_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_use_double_quant=True,
bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
MODEL_NAME = "lmsys/vicuna-7b-v1.5"
model = AutoModelForCausalLM.from_pretrained(
MODEL_NAME,
quantization_config=nf4_config,
low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

##Tạo pipeline

In [ ]:
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto",
)

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(
    pipeline=model_pipeline,
    model_kwargs=gen_kwargs
)

#Tạo prompt

In [ ]:
custom_prompt_template = """Sử dụng tất cả các thông tin sau đây để trả lời câu hỏi của người dùng.
Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời,
hãy đảm bảo giải thích một cách đầy đủ, chi tiết, mạch lạc, có logic, sử dụng toàn bộ tài liệu liên quan.
Lưu ý: tránh tối đa việc lặp lại thông tin.
Tất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt.

Context: {context}
Question: {question}
Answer:
"""


In [ ]:
from langchain import PromptTemplate
def set_custom_prompt():
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt


In [ ]:
prompt = set_custom_prompt()

In [ ]:
print(prompt.template)

Sử dụng tất cả các thông tin sau đây để trả lời câu hỏi của người dùng.
Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời,
hãy đảm bảo giải thích một cách đầy đủ, chi tiết, mạch lạc, có logic, sử dụng toàn bộ tài liệu liên quan.
Lưu ý: tránh tối đa việc lặp lại thông tin.
Tất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt.

Context: {context}
Question: {question}
Answer:



#Kết quả

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "Nội dung đường lối kháng chiến chống thực dân Pháp xâm lược 1946 - 1954?"
output = rag_chain.invoke(question)
output

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'Sử dụng tất cả các thông tin sau đây để trả lời câu hỏi của người dùng.\nNếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời,\nhãy đảm bảo giải thích một cách đầy đủ, chi tiết, mạch lạc, có logic, sử dụng toàn bộ tài liệu liên quan.\nLưu ý: tránh tối đa việc lặp lại thông tin.\nTất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt.\n\nContext: Trình bày ý b, c câu 4.1.\nV Đường lối kháng chiến chống thực dân Pháp xâm lược 1946 -\n1954\nCâu 5.1\nĐường lối kháng chiến chống thực dân Pháp xâm lược 1946 - 1954?\na. Nguyên nhân bùng nổ của cuộc kháng chiến\nVới dã tâm xâm lược nước ta một lần nữa, Pháp đã có những hành động trắng trợn vi phạm các điều\nđã ký kết với chính phủ ta như Hiệp định sơ bộ (06/03/1954), Tạm ước (14/09/1946).\nMặc dù chúng ta đã nhân nhượng với Pháp nhưng chúng ta càng nhân nhượng Pháp càng lấn tới, đến\nlúc chúng ta không thể tiếp tục nhân nhượng, toàn Đảng toàn dân ta quyết tâm kháng chiến để bảo vệ\nnền độc lập dân t

In [ ]:
answer = output.split("Answer:")[1].strip()
print(answer)

* Cơ sở của đường lối: Đường lối kháng chiến chống thực dân Pháp xâm lược 1946 - 1954 được thiết lập với mục đích kháng chiến chống lại sự xâm lược của Pháp và bảo vệ độc lập dân tộc của Việt Nam.
* Mục đích kháng chiến: Đường lối này có mục đích kháng chiến chống lại sự xâm lược của Pháp và bảo vệ độc lập dân tộc của Việt Nam.
* Tính chất của cuộc kháng chiến: Cuộc kháng chiến này là một cuộc kháng chiến toàn quốc, được thực hiện bởi toàn dân và toàn diện, với sự lâu dài và dựa vào sức mình là chính.
* Phương châm đường lối kháng chiến: Toàn dân, toàn diện, lâu dài, dựa vào sức mình là chính. Trong đó: Kháng chiến toàn dân là... Kháng chiến toàn diện là... Kháng chiến lâu dài là... Kháng chiến tự lực cánh sinh, dựa vào sức mình là chính là...
* Kết luận: Đường lối kháng chiến chống thực dân Pháp xâm lược 1946 - 1954 là một đường lối quan trọng trong lịch sử Việt Nam, được thiết lập để kháng chiến chống lại sự xâm lược của Pháp và bảo vệ độc lập dân tộc của Việt Nam.
